In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import warnings
warnings.filterwarnings('ignore')

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [6]:
raw_df = pd.read_gsheet("/content/drive/My Drive/Colab Notebooks/Football Prediction/model_data_18-19.gsheet")
raw_df

AttributeError: ignored

In [ ]:
raw_df.columns

Index(['Attendance', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HS', 'AS',
       'HomeTeamP', 'AwayTeamP', 'FTR_Home'],
      dtype='object')

In [ ]:
#Let's keep aside the attendance data for now
df = raw_df[['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'HS', 'AS',
       'HomeTeamP', 'AwayTeamP', 'FTR_Home']]

## Feature Engineering

In [ ]:
#Let's feature engineer goal conversion rate features for home and away teams
#Let's name it as HTGCR & ATGCR
df['HomeTeamGCR'] = df['FTHG']/df['HS'] 
df['AwayTeamGCR'] = df['FTAG']/df['AS'] 

In [ ]:
df = df.drop(columns = ['FTHG', 'FTAG', 'HS', 'AS'])
df

,HomeTeam,AwayTeam,HomeTeamP,AwayTeamP,FTR_Home,HomeTeamGCR,AwayTeamGCR
0,Manchester United,Leicester City,0.44,0.56,2,0.222222,0.071429
1,Newcastle United,Tottenham Hotspur,0.44,0.56,0,0.062500,0.117647
2,AFC Bournemouth,Cardiff City,0.62,0.38,2,0.222222,0.000000
3,Watford,Brighton & Hove Albion,0.56,0.44,2,0.105263,0.000000
4,Fulham,Crystal Palace,0.60,0.40,0,0.000000,0.166667
...,...,...,...,...,...,...,...
375,Manchester United,Cardiff City,0.60,0.40,0,0.000000,0.142857
376,Liverpool,Wolverhampton Wanderers,0.60,0.40,2,0.142857,0.000000
377,Watford,West Ham United,0.47,0.53,0,0.055556,0.266667
378,Fulham,Newcastle United,0.61,0.39,0,0.000000,0.266667


In [ ]:
df['{}GCR'.format('AwayTeam')].sum()

40.89827264754509

In [ ]:
def Performance(df, location, team_name, n):
    for arena in location:
        for team in team_name:
            index = []
            for idx,row in df.iterrows():
                if(row[arena]==team):
                    index.append(idx)
            #Creating a dataframe out of the index values of a particular team
            matches = df.iloc[index]
            #Calculating Simple Moving Average of GCR from past n games
            matches['{}SMAGCR'.format(arena)] = matches.loc[:,'{}GCR'.format(arena)].rolling(window=n).mean()
            #Shot performance (SP) until last match will be feature for next match
            matches['{}SP'.format(arena)] = matches['{}SMAGCR'.format(arena)].shift(1)
            #
            #Calculating Simple Moving Average of Possession from past n games
            matches['{}SMAP'.format(arena)] = matches.loc[:,'{}P'.format(arena)].rolling(window=n).mean()
            #Hence possession performance (PP) until last match will be feature for next match
            matches['{}PP'.format(arena)] = matches['{}SMAP'.format(arena)].shift(1)
            df = matches.combine_first(df)
    return df

In [ ]:
df.HomeTeam.nunique()

20

In [ ]:
locations = ['HomeTeam', 'AwayTeam']
team_names = df.HomeTeam.unique()
team_names

array(['Manchester United', 'Newcastle United', 'AFC Bournemouth',
       'Watford', 'Fulham', 'Huddersfield Town',
       'Wolverhampton Wanderers', 'Southampton', 'Liverpool', 'Arsenal',
       'Cardiff City', 'Tottenham Hotspur', 'Leicester City', 'Everton',
       'West Ham United', 'Chelsea', 'Manchester City', 'Burnley',
       'Brighton & Hove Albion', 'Crystal Palace'], dtype=object)

In [ ]:
df_model = Performance(df, locations, team_names, 2)

In [ ]:
df_model.columns

Index(['AwayTeam', 'AwayTeamGCR', 'AwayTeamP', 'AwayTeamPP', 'AwayTeamSMAGCR',
       'AwayTeamSMAP', 'AwayTeamSP', 'FTR_Home', 'HomeTeam', 'HomeTeamGCR',
       'HomeTeamP', 'HomeTeamPP', 'HomeTeamSMAGCR', 'HomeTeamSMAP',
       'HomeTeamSP'],
      dtype='object')

In [ ]:
df_model = df_model[['HomeTeam', 'HomeTeamPP', 'HomeTeamSP',
                    'AwayTeam', 'AwayTeamPP', 'AwayTeamSP',
                    'FTR_Home']]
pd.set_option('display.max_rows', None)                    
df_model

,HomeTeam,HomeTeamPP,HomeTeamSP,AwayTeam,AwayTeamPP,AwayTeamSP,FTR_Home
0,Manchester United,NaN,NaN,Leicester City,NaN,NaN,2.0
1,Newcastle United,NaN,NaN,Tottenham Hotspur,NaN,NaN,0.0
2,AFC Bournemouth,NaN,NaN,Cardiff City,NaN,NaN,2.0
3,Watford,NaN,NaN,Brighton & Hove Albion,NaN,NaN,2.0
4,Fulham,NaN,NaN,Crystal Palace,NaN,NaN,0.0
5,Huddersfield Town,NaN,NaN,Chelsea,NaN,NaN,0.0
6,Wolverhampton Wanderers,NaN,NaN,Everton,NaN,NaN,1.0
7,Southampton,NaN,NaN,Burnley,NaN,NaN,1.0
8,Liverpool,NaN,NaN,West Ham United,NaN,NaN,2.0
9,Arsenal,NaN,NaN,Manchester City,NaN,NaN,0.0


In [ ]:
pd.set_option('display.max_rows', 20)
df_model.dropna(inplace=True)
df_model.reset_index(drop=True, inplace=True)

In [ ]:
df_model

,HomeTeam,HomeTeamPP,HomeTeamSP,AwayTeam,AwayTeamPP,AwayTeamSP,FTR_Home
0,Watford,0.515,0.124060,Tottenham Hotspur,0.520,0.195187,2.0
1,Manchester City,0.730,0.130909,Fulham,0.435,0.140909,2.0
2,AFC Bournemouth,0.550,0.173611,Leicester City,0.505,0.160714,2.0
3,Chelsea,0.610,0.101667,Cardiff City,0.415,0.000000,2.0
4,Newcastle United,0.355,0.102679,Arsenal,0.540,0.142157,0.0
...,...,...,...,...,...,...,...
332,Manchester United,0.470,0.071429,Cardiff City,0.385,0.090909,0.0
333,Liverpool,0.640,0.176136,Wolverhampton Wanderers,0.555,0.112745,2.0
334,Watford,0.545,0.081250,West Ham United,0.485,0.052632,0.0
335,Fulham,0.520,0.132479,Newcastle United,0.410,0.091667,0.0


In [ ]:
df_model.shape

(337, 7)

In [ ]:
df_model = pd.get_dummies(df_model, columns=['HomeTeam', 'AwayTeam'])

In [ ]:
df_model.shape

(337, 45)

In [ ]:
df_model.columns

Index(['HomeTeamPP', 'HomeTeamSP', 'AwayTeamPP', 'AwayTeamSP', 'FTR_Home',
       'HomeTeam_AFC Bournemouth', 'HomeTeam_Arsenal',
       'HomeTeam_Brighton & Hove Albion', 'HomeTeam_Burnley',
       'HomeTeam_Cardiff City', 'HomeTeam_Chelsea', 'HomeTeam_Crystal Palace',
       'HomeTeam_Everton', 'HomeTeam_Fulham', 'HomeTeam_Huddersfield Town',
       'HomeTeam_Leicester City', 'HomeTeam_Liverpool',
       'HomeTeam_Manchester City', 'HomeTeam_Manchester United',
       'HomeTeam_Newcastle United', 'HomeTeam_Southampton',
       'HomeTeam_Tottenham Hotspur', 'HomeTeam_Watford',
       'HomeTeam_West Ham United', 'HomeTeam_Wolverhampton Wanderers',
       'AwayTeam_AFC Bournemouth', 'AwayTeam_Arsenal',
       'AwayTeam_Brighton & Hove Albion', 'AwayTeam_Burnley',
       'AwayTeam_Cardiff City', 'AwayTeam_Chelsea', 'AwayTeam_Crystal Palace',
       'AwayTeam_Everton', 'AwayTeam_Fulham', 'AwayTeam_Huddersfield Town',
       'AwayTeam_Leicester City', 'AwayTeam_Liverpool',
       'AwayTeam

In [ ]:
inputs = df_model.drop(columns=['FTR_Home'])

In [ ]:
target = df_model[['FTR_Home']]

In [ ]:
inputs.shape

(337, 44)

In [ ]:
target.shape

(337, 1)

### Test Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(inputs, target, test_size=0.025, shuffle = True, random_state = 18)

In [ ]:
X_train.shape

(328, 44)

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
X_test.shape

(9, 44)

In [ ]:
y_train.shape

(328, 1)

In [ ]:
y_test.shape

(9, 1)

## Model Selection

In [ ]:
model_params = {
    'logistic_regression' : {
        'model': LogisticRegression(multi_class = 'multinomial'),
        'params': {
            'penalty': ['l1','l2', 'elasticnet'],
            'C': [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1, 2],
            'solver': ['newton-cg','saga','sag','lbfgs']
        }
    },
    'svm': {
        'model': SVC(),
        'params' : {
            'C': [0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1, 2],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': ['scale', 'auto'],
            'decision_function_shape': ['ovo', 'ovr']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'criterion': ['gini','entropy'],
            'n_estimators': [80, 100, 120, 150, 200,400]
        }
    },  
}

In [ ]:
scores = []

for model_name, mp in model_params.items():
    model =  GridSearchCV(mp['model'], mp['params'], cv=30, return_train_score=False)
    model.fit(X_train, y_train)
    scores.append({
        'model': model_name,
        'best_score': model.best_score_,
        'best_params': model.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,logistic_regression,0.572381,"{'C': 0.25, 'penalty': 'l1', 'solver': 'saga'}"
1,svm,0.568254,"{'C': 0.75, 'decision_function_shape': 'ovo', ..."
2,random_forest,0.555556,"{'criterion': 'entropy', 'n_estimators': 200}"


In [ ]:
scores[0]['best_params']

{'C': 0.25, 'penalty': 'l1', 'solver': 'saga'}

In [ ]:
scores[1]['best_params']

{'C': 0.75,
 'decision_function_shape': 'ovo',
 'gamma': 'scale',
 'kernel': 'poly'}

In [ ]:
scores[2]['best_params']

{'criterion': 'entropy', 'n_estimators': 200}